# Dipole autoencoding
Testing autoencoding of simulated current dipoles

In [1]:
import torch
from torch import nn
import dill
import os.path as op
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import multiprocessing
import pandas as pd
import sys
sys.path.append('../')
import spike_train_functions
import hnn_simnets_functions
import numba
import sbi_functions
import umap
import umap.plot
import multiprocessing
num_cores = multiprocessing.cpu_count()
use_cuda = torch.cuda.is_available()
#device = torch.device("cuda:0")
device = torch.device("cpu")

/home/ntolley/anaconda3/lib/python3.7/site-packages/elephant/pandas_bridge.py:22: DeprecationWarning: pandas_bridge module will be removed in Elephant v0.8.x
  DeprecationWarning)
/home/ntolley/anaconda3/lib/python3.7/site-packages/holoviews/operation/datashader.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Callable


In [2]:
save_name = 'beta_event_expanse_t100000_02142021_182526'
data_path = '/home/ntolley/Jones_Lab/sbi_hnn/data/beta/prerun_simulations/' + save_name + '/'
dpl_all = np.loadtxt(data_path + 'dpl_100x_downsample.csv', delimiter=',')
theta_all = np.loadtxt(data_path + 'theta_all.csv', delimiter=',')
sim_length = 500 # in ms
sim_times_all = np.linspace(0,sim_length, dpl_all.shape[1])


In [3]:
# Parameters
batch_size = 500
train_params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': num_cores, 'pin_memory':False}


# Generators
training_set = sbi_functions.HNNDataset(dpl_all, theta_all, device)
training_generator = torch.utils.data.DataLoader(training_set, **train_params)


In [4]:
# Define hyperparameters
lr=1e-3
weight_decay=1e-2
dropout=0.8
window_size=10
input_size = 1
hidden_size = 10
step_size = 1
latent_dim = 3
layer_size = [10,latent_dim]
max_epochs=1000

#Define model
model =  sbi_functions.autoencoder_gru(input_size=input_size, hidden_size=hidden_size, layer_size=layer_size, window_size=window_size, step_size=step_size, dropout=dropout, device=device).to(device)

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)



In [5]:
train_loss_array = []
model.train()
# Loop over epochs
for epoch in range(max_epochs):
    train_batch_loss = []
    for batch, _ in training_generator:
        optimizer.zero_grad() # Clears existing gradients from previous epoch
        batch = batch.float().to(device)
        print(batch.shape)
        output, latent = model(batch)
        train_loss = criterion(output, batch[:,-1,:])
        train_loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly

        train_batch_loss.append(train_loss.item())
        print('*',end='')
    train_loss_array.append(train_batch_loss)
    #Print Loss
    if (epoch+1)%1 == 0:
        print('')
        print('Epoch: {}/{} ...'.format(epoch+1, max_epochs), end=' ')
        print('Train Loss: ' + str(np.mean(train_batch_loss)))



torch.Size([500, 101, 1])


RuntimeError: input.size(-1) must be equal to input_size. Expected 101, got 1